<a href="https://colab.research.google.com/github/LucasASouzak/AluraImersao-GoogleGemini/blob/main/Alura%2BGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -U -q google-generativeai


In [ ]:
!pip install -q google-auth
!pip install -U -q google-generativeai
!pip install -q google-cloud-secret-manager

# Importações e configurações iniciais

In [ ]:
!pip install -q google-api-python-client
from googleapiclient.discovery import build

service = build('sheets', 'v4')
# Importa confirgurações iniciais do Sheets

In [ ]:
import gspread
import numpy as np
import pandas as pd
import google.generativeai as genai
import json
from google.colab import auth
auth.authenticate_user()
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = "SEU_PROJECT_ID" # Substitua pelo ID do seu projeto

In [ ]:
from google.cloud import secretmanager
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
GOOGLE_API_KEY = "SUA_API_KEY" # Substitua pela sua API Key do Gemini
genai.configure(api_key=GOOGLE_API_KEY)
projeto_id = "SEU_PROJECT_ID"
nome_do_segredo = "projeto_desaparecidos" # Nome do segredo no Secret Manager

In [ ]:
cliente_secret = secretmanager.SecretManagerServiceClient()
nome = f"projects/{projeto_id}/secrets/{nome_do_segredo}/versions/latest"
resposta = cliente_secret.access_secret_version(name=nome)
chave_json = resposta.payload.data.decode("UTF-8")


In [ ]:
def acessar_planilha():
    escopo = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(chave_json), escopo)
    cliente = gspread.authorize(creds)
    planilha = cliente.open_by_url('https://docs.google.com/spreadsheets/d/1pr1jjq3rU5-UOdh-p4niJ62OZeImQ0zqLGne7wQZBmM/edit')
    return planilha.worksheet('Lista de Pessoas Desaparecidas')

In [ ]:
def adicionar_pessoa(nome, idade, genero, cidade, filiacao, rg, celular):
    planilha = acessar_planilha()
    nova_linha = [nome, idade, genero, cidade, filiacao, rg, celular]
    planilha.append_row(nova_linha)

In [ ]:
def consultar_pessoa(nome, idade, genero, cidade, filiacao, rg):
    planilha = acessar_planilha()
    dados = planilha.get_all_records()
    for pessoa in dados:
        if (pessoa['Nome'] == nome and
                pessoa['Idade'] == int(idade) and
                pessoa['Genero'] == genero and
                pessoa['Cidade'] == cidade and
                pessoa['Filiação'] == filiacao and
                pessoa['Rg'] == rg):
            return pessoa  # Retorna a linha da planilha com os dados da pessoa
    return None

In [ ]:
def coletar_informacoes():
    print("Olá, sou o Gemini. Para te ajudar, preciso de algumas informações sobre a pessoa desaparecida.")
    nome = input("Digite o nome da pessoa: ")
    idade = input("Digite a idade: ")
    genero = input("Digite o gênero: ")
    cidade = input("Digite a cidade: ")
    filiacao = input("Digite a filiação: ")
    rg = input("Digite o RG: ")
    return nome, idade, genero, cidade, filiacao, rg


In [ ]:
# Fluxo principal
nome, idade, genero, cidade, filiacao, rg = coletar_informacoes()
pessoa_encontrada = consultar_pessoa(nome, idade, genero, cidade, filiacao, rg)

if pessoa_encontrada:
    print(f"Temos informações sobre {nome}.")
    print(f"Ele(a) está no abrigo: {pessoa_encontrada['Abrigo']}")
    print(f"Telefone do abrigo: {pessoa_encontrada['Telefone do Abrigo']}")
else:
    generation_config = {
      "candidate_count": 1,
      "temperature": 0.5,
    }
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_NONE',
        'DANGEROUS' : 'BLOCK_NONE'
    }
    model = genai.GenerativeModel(
        model_name='gemini-1.0-pro',
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    prompt= f"Crie uma mensagem de conforto para alguém que está procurando por {nome}, {idade} anos, {genero}, de {cidade}."
    response = model.generate_content(contents=[prompt])
    print(response.candidates[0].content)
    celular = input("Digite um número de celular para contato: ")
    adicionar_pessoa(nome, idade, genero, cidade, filiacao, rg, celular)
    print(f"As informações foram adicionadas à planilha. Esperamos que {nome} seja encontrado(a) em breve.")


Olá, sou o Gemini. Para te ajudar, preciso de algumas informações sobre a pessoa desaparecida.
Digite o nome da pessoa: Lucas Antonio de Souza
Digite a idade: 33
Digite o gênero: Masculino
Digite a cidade: São Paulo 
Digite a filiação: José Augusto
Digite o RG: 447285695
parts {
  text: "Querida pessoa que busca por Lucas Antonio de Souza,\n\nEntendo profundamente sua preocupa\303\247\303\243o e ang\303\272stia enquanto voc\303\252 procura por seu ente querido. Quero oferecer minhas sinceras condol\303\252ncias e apoio durante este momento dif\303\255cil.\n\nCompreendo que a incerteza e a falta de informa\303\247\303\265es podem ser assustadoras. Saiba que voc\303\252 n\303\243o est\303\241 sozinho e que h\303\241 pessoas que se preocupam com voc\303\252 e com o bem-estar de Lucas.\n\nRecomendo fortemente que voc\303\252 entre em contato com as autoridades locais e registre um boletim de ocorr\303\252ncia sobre o desaparecimento de Lucas. Forne\303\247a o m\303\241ximo de detalhes poss